In [1]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import pandas as pd
from openpyxl import load_workbook
import re
import io
import os
import pdb

import sys
sys.path.append(os.path.abspath('../kev/'))

import eq_data
import eq_preproc
import eq_evaluator
import eq_postproc
import eq_writer


# basic input ------------------------------------------------
 
def cl_scripts_load(_sep, _subdir, _file):
    
    # if specific file selected it should be XLSX one
    if _file != "":
        
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        _file = _subdir + _file
        
        # open excel file
        with open(_file, "rb") as f:
            inmemory_file = io.BytesIO(f.read())
        wb = load_workbook(inmemory_file, read_only = True)
                
        r = re.compile(r'heats')
        heats_data = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], index_col = 0)
        
        r = re.compile(r'targets')
        targets = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], header = None)
        
        r = re.compile(r'enthalpies')
        delta_H = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0])
        
    # use a bunch of plain text files instead
    else:
          
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        file_names = list(os.listdir(path = _subdir))

        r = re.compile(r'heats')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        heats_data = pd.read_csv(file, sep = _sep, decimal = ",", index_col = 0)
        
        r = re.compile(r'targets')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        targets = pd.read_csv(file, sep = _sep, header = None)
        
        r = re.compile(r'enthalpies')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        delta_H = pd.read_csv(file, sep = _sep)
        
    return heats_data, targets, delta_H

In [2]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import numpy as np
import re

# basic preprocessing ----------------------------------------
    
def cl_preproc(con_data, prod_names, idx, type_con, heats_data, targets, delta_H, _calor_setup_type):
    
    # checking if there are several series
    
    if 'series' not in con_data.columns:        
        con_data['series'] = ''
    
    if 'series' not in heats_data.index:
        heats_data.loc['series'] = '' # probably not needed
    
    if 'volumes' not in heats_data.index:
        heats_data.loc['volumes'] = 1        

    # series variables
    
    ser_info = con_data['series'].to_numpy()
    ser_unique = np.unique(ser_info)
    ser_num = np.shape(np.unique(ser_info))[0]
    
    ser_counts = con_data.groupby(['series']).size().tolist();
    
    # reading volumes from experimental data
    volumes = heats_data.loc['volumes'].to_numpy()
    
    # reading exp heats from experimental data
    heats = heats_data.loc['observation'].to_numpy() - heats_data.loc['dilution'].to_numpy()
        
    # transforming deviation to the matrix form
    devs = heats_data.loc['deviation'].to_numpy()
    devs = np.diag((1 / (np.array(devs) * np.array(devs))) * (np.sum(np.array(devs) * np.array(devs)) / len(devs)))
    
    # creating vector of known enthalpies
    dH_par = np.hstack((np.zeros(len(prod_names_con)), np.transpose(delta_H.drop('Reaction', axis = 1).to_numpy())[0]))
    
    # number of constant to find
    tar_names = set(targets.to_numpy()[0][1:])
    tar_num = [index for index, item in enumerate(prod_names) if item in tar_names]
    
    # number of enthalpy to find
    dH_names = np.hstack((prod_names_con, np.transpose(delta_H.drop('Value', axis = 1).to_numpy())[0]))
    un_el = set(prod_names) - set(dH_names)
    dH_ind_wtf = list([index for index, item in enumerate(prod_names) if item in un_el])
    un_el = set(prod_names) - (set(prod_names) - set(dH_names))
    dH_ind = list([index for index, item in enumerate(prod_names) if item in un_el])
    
    # creating array of diff total concentrations
    con_matrix_red = []
    for s in range(ser_num):
        for k in range(ser_counts[s] - 1):
            con_matrix_red.append(con_matrix[int(np.sum(ser_counts[:s])) + k + 1])
            
    if _calor_setup_type == 'DSC' or 'overfilled':
        _calor_setup_type_coeff = [[1] for i in range(len(con_matrix_red))]
    
    if _calor_setup_type == 'ampoule':
        _calor_setup_type_coeff = 1 / np.array(con_matrix_red)[:, idx] 
    
    return (ser_num, ser_counts, ser_info, volumes, heats, devs, tar_num, dH_ind, dH_ind_wtf, dH_par, 
            _calor_setup_type_coeff)

In [3]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import numpy as np

def cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type):

    # finding the increment in equilibrium concentrations of products - needed for dH calculation
    vol_active = 0.988 # terrible solution. Gotta introduce new variable for operating volume 
    vol_init = 15      # of calorimetric cell for overfilled mode
    dif_conc = []        
             
    for s in range(ser_num):    
        for k in range(ser_counts[s] - 1):
            
            if _calor_setup_type == 'overfilled':
                factor = 1 - (volumes[int(np.sum(ser_counts[:s])) + k - s] / vol_active)
            else:
                factor = 1
            
            dif_conc.append(c_res_out[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out[int(np.sum(ser_counts[:s])) +
                                                                                       k] * factor)
            
    dif_conc, devs, volumes = np.array(dif_conc).astype(np.float32), devs.astype(np.float32), volumes.astype(np.float32)
    
    dif_vol = np.diff(volumes)
    dif_vol = np.array(np.hstack((volumes[0] - vol_init, dif_vol)))
    
    # finding vector of experimental values
    
    q_corr = deepcopy(heats)
    
    for i in range(len(heats)):
        for j in range(len(volumes[dH_ind])):
            
            if _calor_setup_type == 'overfilled':
                q_corr[i] -= vol_active * dif_conc[i, dH_ind[j]] * dH_par[j]
            else:
                q_corr[i] -= (volumes[dH_ind[j]] * dif_conc[i, dH_ind[j]] * dH_par[j] +
                              dif_vol[i] * np.array(c_res_out)[i, dH_ind[j]] * dH_par[j])
    
    if _calor_setup_type == 'overfilled' or _calor_setup_type == 'DSC':
        c_eq = []
        for s in range(ser_num):
            for k in range(ser_counts[s] - 1):
                c_eq.append(c_res_out[int(np.sum(ser_counts[:s])) + k])
    else:
        c_eq = dif_conc
    
    if _calor_setup_type == 'overfilled':
        A = (vol_active * dif_conc[:, dH_ind_wtf]) * np.array(_calor_setup_type_coeff)
    
    else:
        A = ((np.array(volumes).reshape(len(volumes), 1) * dif_conc[:, dH_ind_wtf]) * np.array(_calor_setup_type_coeff) +
             dif_vol.reshape((len(dif_vol), 1)) * np.array(c_eq)[:, dH_ind_wtf])
    
    AT = np.transpose(A)
    LTP = np.linalg.inv(np.dot(np.dot(AT, devs), A))
    RTP = np.dot(np.dot(AT, devs), q_corr)
    dH_res = np.dot(LTP, RTP)
    dH_res_tmp = dH_par

    for i in range(len(dH_ind_wtf)):
        dH_res_tmp = np.insert(dH_res_tmp, dH_ind_wtf[i], dH_res[i])

    dH_res_out = dH_res_tmp

    F_min, heats_calc = 0, []
    
    for i in range(len(heats)): 
        
        if _calor_setup_type == 'overfilled':
            F_min += ((vol_active * np.sum(dH_res_out * dif_conc[i] * np.array(_calor_setup_type_coeff[i])) - heats[i])) ** 2
            
            heats_calc.append(np.sum(dH_res_out * dif_conc[i] * vol_active * _calor_setup_type_coeff[i]))
        
        else:
            F_min += ((np.sum(dH_res_out * dif_conc[i] * volumes[i] * np.array(_calor_setup_type_coeff[i])) +
                       np.sum(dH_res_out * c_eq[i] * dif_vol[i]) - heats[i]) ** 2)
            
            heats_calc.append(np.sum(dH_res_out * dif_conc[i] * volumes[i] * np.array(_calor_setup_type_coeff[i]))
                              + np.sum(dH_res_out * c_eq[i] * dif_vol[i]))
    print('\neq con')
    print(np.array(c_res_out))
    print('\ndif con')
    print(dif_conc)
    print('\nvol')
    print(volumes)
    print('\ndH par')
    print(dH_par)
    print('\nsum of terms')
    print(np.array(heats_calc))
    print(heats)
    
    #print(sum(heats))
    #print(dH_res_out)
    #print(np.sum(dif_conc[i]))
    #print(np.sum(volumes[i]))
    #print(np.sum(np.array(_calor_setup_type_coeff[i])))
    
    # calculating st. dev. of dH_res / checking the quality of linear regression
    # consider making it in separate function
    
    disp = (F_min / (len(heats) - len(dH_ind_wtf)) * (np.diag(LTP))) ** 0.5  
    
    return F_min, dH_res_out, disp

In [4]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

from copy import deepcopy
import eq_evaluator
# import also script cl_enthalpies when needed

# main Hooke-Jeeves procedure --------------------------------
def cl_hooke_jeeves(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type):

    basis_point, lg_k, new_min, c_res_out, disp = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type)
    print(new_min)

    for i in range(max_iter):
        print('\niteration', i, lg_k[tar_num])
        if F_min <= new_min:
        
            if step_init < HJ_eps:
                print("p1 " + str(step_init) + " " + str(HJ_eps))
                break
            
            else: 
                 
                step_init /= 2
            
                basis_point, lg_k, new_min, c_res_out, disp = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num,
                                  st_coeff_matrix, type_con, con_matrix, ign_indices, ser_counts, ser_info, devs,
                                  volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
                print('\nF min more ', new_min, F_min, lg_k[tar_num])
                print(str(step_init))
        else:
            F_min = deepcopy(new_min)
            print('\nF min less', new_min, F_min, lg_k[tar_num])
            print(str(step_init))
            basis_point_n = deepcopy(lg_k)
            lg_k[tar_num] = basis_point[tar_num] + 2 * (lg_k[tar_num] - basis_point[tar_num])

            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                    con_matrix, ign_indices)

            new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                          , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
            print('\nF min less after', new_min, F_min, lg_k[tar_num])
        
            if F_min <= new_min:
                lg_k = deepcopy(basis_point_n)
                print('\nconstant less after', basis_point[tar_num], lg_k[tar_num]) 
            else:
                F_min = new_min

            basis_point, lg_k, new_min, c_res_out, disp = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num,
                                  st_coeff_matrix, type_con, con_matrix, ign_indices, ser_counts, ser_info, devs,
                                  volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
        
            if F_min <= new_min:
            
                print('\nF min p2', new_min, F_min, lg_k[tar_num])
                if step_init < HJ_eps:
                    print("p2")
                    break
            
                else: 
                
                    step_init /= 2
        
            else:

                F_min = deepcopy(new_min)
            
                print('\nF min p3.1', new_min, F_min, lg_k[tar_num])
            
                basis_point_n = deepcopy(lg_k)
                lg_k[tar_num] = basis_point[tar_num] + 2 * (lg_k[tar_num] - basis_point[tar_num])
            
                c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                        con_matrix, ign_indices)
    
                new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                              , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
        
                if F_min <= new_min:
                    lg_k = deepcopy(basis_point_n)
            
                print('\nF min p3.2', new_min, F_min, lg_k[tar_num])
    
    return F_min, lg_k, dH_res_out, c_res_out, disp

# exploratory move function used in main function ------------------------
def cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type):

    basis_point = deepcopy(lg_k)
    
    c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k, 
                                                con_matrix, ign_indices)

    new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                          , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)

    #if F_min > new_min:
    F_min = new_min
    
    for j in range(len(tar_num)): 
        #pdb.set_trace()
        print('input', lg_k[tar_num[j]], F_min)
        
        lg_k[tar_num[j]] = lg_k[tar_num[j]] + step_init
        
        c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices)
        #print('\na1 ', new_min, F_min)
        new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                   _calor_setup_type_coeff, _calor_setup_type)
        print('right', lg_k[tar_num[j]], new_min, F_min)
        #print('\na2 ', new_min, F_min)
        if F_min <= new_min:
            
            lg_k[tar_num[j]] = lg_k[tar_num[j]] - 2 * step_init
            
            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k, 
                                                        con_matrix, ign_indices)
        
            new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                                  , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff, _calor_setup_type)
            print('left', lg_k[tar_num[j]], new_min, F_min)
            #print('\na3 ', new_min, F_min)
            if F_min <= new_min:
                
                lg_k[tar_num[j]] = lg_k[tar_num[j]] + step_init
                new_min = deepcopy(F_min)
                #print('\na4 ', new_min, F_min)
            #else:
                
                #new_min = deepcopy(F_min)
                
        #else:
            
            #new_min = deepcopy(F_min)
    print('return', basis_point[tar_num[j]], lg_k[tar_num[j]])
    #print('\na5 ', new_min, F_min)
    return basis_point, lg_k, new_min, c_res_out, disp

In [5]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------
import numpy as np
import eq_evaluator
# import also script cl_enthalpies when needed

def cl_constants_errs(lg_k, heats, tar_num, HJ_eps, max_iter, eps, st_coeff_matrix, type_con, con_matrix,
                      ign_indices, ser_num, ser_counts, devs, volumes, dH_ind, dH_par, dH_ind_wtf, 
                      _calor_setup_type_coeff, _calor_setup_type):
    
    # creating constants vectors ---------------------------------
    lg_k_1, lg_k_2 = np.array(lg_k), np.array(lg_k)
    heats_one, heats_two = [0] * len(heats), [0] * len(heats)
    jc_mtrx = [[0] * len(heats) for i in range(len(tar_num))]

    for i in range(len(tar_num)):
        # calculating the heats for Jacobi matrix computing
        lg_k_1[tar_num[i]] += HJ_eps
        lg_k_2[tar_num[i]] -= HJ_eps
    
        # if lg_k is overestimated
        c_res_out_dist, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k_1,
                                                        con_matrix, ign_indices)
    
        new_min, dH_res_out_dist, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type)
    
        dif_conc = [] 
        for s in range(ser_num):    
            for k in range(ser_counts[s] - 1):
                dif_conc.append(c_res_out_dist[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out_dist[int(np.sum(ser_counts[:s])) + k])
    
        for o in range(len(heats)):
            heats_one[o] = np.sum(dif_conc[o] * dH_res_out_dist) * volumes[o]
    
        # if lg_k is underestimated    
        c_res_out_dist, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k_2,
                                                        con_matrix, ign_indices)
    
        new_min, dH_res_out_dist, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type)
    
        dif_conc = [] 
        for s in range(ser_num):    
            for k in range(ser_counts[s] - 1):
                dif_conc.append(c_res_out_dist[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out_dist[int(np.sum(ser_counts[:s])) + k])    
   
        for o in range(len(heats)):
            heats_two[o] = np.sum(dif_conc[o] * dH_res_out_dist) * volumes[o]
    
    # calculating the Jacobi matrix    
    jc_mtrx[i] = (np.array(heats_one) - np.array(heats_two)) / (2 * HJ_eps)

    jc_mtrx, devs = np.array(jc_mtrx).astype(np.float32), devs.astype(np.float32)

    cov_matrix = (F_min / (len(heats) - len(tar_num))) * np.linalg.inv(np.dot(np.dot(jc_mtrx, devs), np.transpose(jc_mtrx)))

    errs_values = np.diag(cov_matrix) ** 0.5
    return errs_values

In [6]:
# input for xlsx file
_subdir = "calorimetry"
_sep = ";"
_file = "test_5.xlsx"
_calor_setup_type = 'DSC' # use 'DSC' for test_1, test_2 and test_5; use 'ampoule' for test_3; use 'overfilled' for test_4

max_iter, eps, HJ_eps, step_init = 100, 0.0000001, 0.0001, 0.5

# load data

st_coeff_data, lg_k_data, con_data, type_con, component_name_for_yields = eq_data.eq_scripts_load(_sep, _subdir, _file)
heats_data, targets, delta_H = cl_scripts_load(_sep, _subdir, _file)

# preproc

st_coeff_matrix, prod_names, lg_k, prod_names_con, con_matrix, ign_indices, idx, type_con = eq_preproc.eq_preproc(
    st_coeff_data, con_data, type_con, lg_k_data, component_name_for_yields)

(ser_num, ser_counts, ser_info, volumes, heats, devs, 
 tar_num, dH_ind, dH_ind_wtf, dH_par, _calor_setup_type_coeff) = cl_preproc(
    con_data, prod_names, idx, type_con, heats_data, targets, delta_H, _calor_setup_type)
#----
#lg_k[tar_num] = 10 # it is just for quick variation of initial approximation of the constant to be optimized. Delete in the end
#----

# calculation of eq concentration (starting point)
c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices)

# calculation of F_min and unknown enthalpies (starting point)
F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               _calor_setup_type_coeff, _calor_setup_type)

# F_min assignment for further calculations
new_min = F_min

# constant refinment
if len(tar_num) > 0:
    F_min, lg_k, dH_res_out, c_res_out, disp = cl_hooke_jeeves(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff, _calor_setup_type)


F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               _calor_setup_type_coeff, _calor_setup_type)

# error estimation
#if len(tar_num) > 0:
    #errs_values = cl_constants_errs(lg_k, heats, tar_num, HJ_eps, max_iter, eps, st_coeff_matrix, type_con, con_matrix,
                                    #ign_indices, ser_num, ser_counts, devs, volumes, dH_ind, dH_par, dH_ind_wtf, 
                                    #_calor_setup_type_coeff, _calor_setup_type)


eq con
[[5.00000006e-17 1.00000000e-03 5.00000006e-17 2.50000006e-31]
 [8.60140605e-05 9.18640463e-04 7.90159963e-05 6.79648669e-07]
 [1.77254089e-04 8.44358803e-04 1.49666050e-04 2.65289194e-06]
 [2.73467516e-04 7.76789118e-04 2.12426591e-04 5.80917722e-06]
 [3.74349759e-04 7.15502450e-04 2.67848170e-04 1.00268898e-05]
 [4.79561977e-04 6.60030596e-04 3.16525578e-04 1.51793632e-05]
 [5.88748060e-04 6.09888268e-04 3.59070534e-04 2.11402080e-05]
 [7.01548795e-04 5.64591699e-04 3.96088626e-04 2.77875498e-05]
 [8.17613281e-04 5.23673006e-04 4.28162005e-04 3.50070941e-05]
 [9.36606978e-04 4.86690193e-04 4.55837431e-04 4.26940518e-05]
 [1.05821690e-03 4.53233305e-04 4.79619144e-04 5.07541084e-05]
 [1.18215443e-03 4.22927479e-04 4.99965593e-04 5.91036541e-05]
 [1.30815631e-03 3.95433646e-04 5.17289021e-04 6.76694899e-05]
 [1.43598439e-03 3.70447624e-04 5.31957004e-04 7.63881953e-05]
 [1.56542446e-03 3.47698171e-04 5.44295222e-04 8.52053055e-05]
 [1.69628469e-03 3.26944465e-04 5.54590892e-04 

 0.00150741 0.00146997]
[0.02239 0.02159 0.02042 0.01878 0.01661 0.01412 0.01167 0.00961 0.00805
 0.00693 0.00611 0.00551 0.00506 0.0047 0.00442 0.00418 0.00397 0.0038
 0.00364 0.0035]
right [4.5] 0.00013244010597285668 4.532356189772895e-10

eq con
[[2.40253075e-17 1.00000000e-03 7.59746932e-17 5.77215400e-31]
 [4.32341312e-05 8.76819828e-04 1.19877340e-04 1.63894294e-06]
 [9.33214316e-05 7.64430601e-04 2.25589799e-04 6.65734171e-06]
 [1.50844539e-04 6.63453242e-04 3.16475369e-04 1.50962649e-05]
 [2.16160573e-04 5.74112011e-04 3.92439863e-04 2.68256096e-05]
 [2.89350848e-04 4.96182439e-04 4.54010763e-04 4.15423353e-05]
 [3.70219783e-04 4.29022802e-04 5.02273190e-04 5.88030184e-05]
 [4.58341208e-04 3.71677287e-04 5.38709864e-04 7.80807242e-05]
 [5.53132719e-04 3.23013075e-04 5.65001305e-04 9.88277252e-05]
 [6.53933706e-04 2.81850321e-04 5.82843901e-04 1.20527453e-04]
 [7.60069478e-04 2.47059339e-04 5.93819655e-04 1.42727566e-04]
 [8.70894709e-04 2.17618526e-04 5.99323788e-04 1.65054424

 0.00436902 0.0041809 ]
[0.02239 0.02159 0.02042 0.01878 0.01661 0.01412 0.01167 0.00961 0.00805
 0.00693 0.00611 0.00551 0.00506 0.0047 0.00442 0.00418 0.00397 0.0038
 0.00364 0.0035]
left [3.875] 1.4685800569617959e-05 4.532356189772895e-10
input [6.] 4.532356189772895e-10

eq con
[[9.09090925e-18 1.00000000e-03 9.09090925e-17 1.10208387e-31]
 [1.74468562e-05 8.49738532e-04 1.48252659e-04 3.44921029e-07]
 [4.07890372e-05 7.06809016e-04 2.88300592e-04 1.56815500e-06]
 [7.26012888e-05 5.74149377e-04 4.16839848e-04 4.03565060e-06]
 [1.16244792e-04 4.55580025e-04 5.29588053e-04 8.20940506e-06]
 [1.75226071e-04 3.55052851e-04 6.22145162e-04 1.45375242e-05]
 [2.51694232e-04 2.74918995e-04 6.91955253e-04 2.32247621e-05]
 [3.45110711e-04 2.14421386e-04 7.39991168e-04 3.40553207e-05]
 [4.52622270e-04 1.70162291e-04 7.70192424e-04 4.64873897e-05]
 [5.70663754e-04 1.37968678e-04 7.87337236e-04 5.99157611e-05]
 [6.96157708e-04 1.14268566e-04 7.95489429e-04 7.38485630e-05]
 [8.26848236e-04 9.6461


eq con
[[9.09090925e-18 1.00000000e-03 9.09090925e-17 7.15673020e-32]
 [1.74738972e-05 8.49645308e-04 1.48466148e-04 2.24655583e-07]
 [4.09432711e-05 7.06420579e-04 2.89231693e-04 1.02548288e-06]
 [7.31065439e-05 5.73272196e-04 4.19099490e-04 2.65322075e-06]
 [1.17553984e-04 4.54114062e-04 5.33829171e-04 5.43425049e-06]
 [1.78114538e-04 3.53104469e-04 6.28930393e-04 9.70067492e-06]
 [2.57189480e-04 2.72816533e-04 7.01655424e-04 1.56270531e-05]
 [3.54198611e-04 2.12545072e-04 7.52831696e-04 2.30911068e-05]
 [4.65972498e-04 1.68755803e-04 7.86355629e-04 3.17306731e-05]
 [5.88585150e-04 1.37105801e-04 8.06984385e-04 4.11314890e-05]
 [7.18700747e-04 1.13918444e-04 8.18732710e-04 5.09554027e-05]
 [8.53916212e-04 9.65524889e-05 8.24477355e-04 6.09668829e-05]
 [9.92609365e-04 8.32300496e-05 8.26149267e-04 7.10128414e-05]
 [1.13370126e-03 7.27726034e-05 8.25023925e-04 8.09962988e-05]
 [1.27646779e-03 6.43925261e-05 8.21949857e-04 9.08563291e-05]
 [1.42041350e-03 5.75537453e-05 8.17501166e-04 


eq con
[[9.09090925e-18 1.00000000e-03 9.09090925e-17 8.26446310e-32]
 [1.74661282e-05 8.49672089e-04 1.48404817e-04 2.59205756e-07]
 [4.08988218e-05 7.06532473e-04 2.88963457e-04 1.18182649e-06]
 [7.29603133e-05 5.73525740e-04 4.18446177e-04 3.05299642e-06]
 [1.17172994e-04 4.54539426e-04 5.32597454e-04 6.24060383e-06]
 [1.77268767e-04 3.53671885e-04 6.26949791e-04 1.11138616e-05]
 [2.55570971e-04 2.73430500e-04 6.98808981e-04 1.78595290e-05]
 [3.51508998e-04 2.13093935e-04 7.49044356e-04 2.63295831e-05]
 [4.62006282e-04 1.69167733e-04 7.81565553e-04 3.61088195e-05]
 [5.83243856e-04 1.37358842e-04 8.01137009e-04 4.67258238e-05]
 [7.11962647e-04 1.14021282e-04 8.11788936e-04 5.77963399e-05]
 [8.45803431e-04 9.65257450e-05 8.16418062e-04 6.90529198e-05]
 [9.83166251e-04 8.30962050e-05 8.16973843e-04 8.03221110e-05]
 [1.12298240e-03 7.25520059e-05 8.14746254e-04 9.14945699e-05]
 [1.26453171e-03 6.41023220e-05 8.10594190e-04 1.02502206e-04]
 [1.40731948e-03 5.72079581e-05 8.05098739e-04 

...

 0.00363759 0.00349803]
[0.02239 0.02159 0.02042 0.01878 0.01661 0.01412 0.01167 0.00961 0.00805
 0.00693 0.00611 0.00551 0.00506 0.0047 0.00442 0.00418 0.00397 0.0038
 0.00364 0.0035]
left [5.99902344] 1.1278048804523456e-09 4.532356189772895e-10
return [6.] [6.]

F min more  4.532356189772895e-10 4.532356189772895e-10 [[4.]
 [6.]]
0.0009765625

iteration 9 [[4.]
 [6.]]

eq con
[[9.09090925e-18 1.00000000e-03 9.09090925e-17 8.26446310e-32]
 [1.74661282e-05 8.49672089e-04 1.48404817e-04 2.59205756e-07]
 [4.08988218e-05 7.06532473e-04 2.88963457e-04 1.18182649e-06]
 [7.29603133e-05 5.73525740e-04 4.18446177e-04 3.05299642e-06]
 [1.17172994e-04 4.54539426e-04 5.32597454e-04 6.24060383e-06]
 [1.77268767e-04 3.53671885e-04 6.26949791e-04 1.11138616e-05]
 [2.55570971e-04 2.73430500e-04 6.98808981e-04 1.78595290e-05]
 [3.51508998e-04 2.13093935e-04 7.49044356e-04 2.63295831e-05]
 [4.62006282e-04 1.69167733e-04 7.81565553e-04 3.61088195e-05]
 [5.83243856e-04 1.37358842e-04 8.01137009e-04 4.67

 0.00364241 0.00350263]
[0.02239 0.02159 0.02042 0.01878 0.01661 0.01412 0.01167 0.00961 0.00805
 0.00693 0.00611 0.00551 0.00506 0.0047 0.00442 0.00418 0.00397 0.0038
 0.00364 0.0035]
input [4.] 4.532356189772895e-10

eq con
[[9.08626442e-18 1.00000000e-03 9.09137374e-17 8.25602010e-32]
 [1.74575617e-05 8.49663265e-04 1.48413897e-04 2.58948866e-07]
 [4.08799756e-05 7.06512505e-04 2.88984548e-04 1.18070420e-06]
 [7.29298044e-05 5.73492502e-04 4.18482145e-04 3.05026690e-06]
 [1.17130852e-04 4.54492147e-04 5.32649870e-04 6.23546701e-06]
 [1.77217841e-04 3.53612716e-04 6.27017203e-04 1.11056187e-05]
 [2.55517083e-04 2.73364791e-04 6.98886511e-04 1.78477082e-05]
 [3.51458670e-04 2.13027913e-04 7.49126074e-04 2.63138887e-05]
 [4.61964135e-04 1.69105776e-04 7.81647320e-04 3.60890095e-05]
 [5.83212022e-04 1.37302869e-04 8.01217121e-04 4.67016848e-05]
 [7.11941558e-04 1.13971574e-04 8.11867263e-04 5.77677209e-05]
 [8.45792686e-04 9.64818318e-05 8.16495144e-04 6.90197514e-05]
 [9.83165137e-04 8

 [ 1.45299215e-04 -3.04690616e-06 -8.07333709e-06  9.55684754e-06]]

vol
[15.024997 15.049994 15.074991 15.099988 15.124985 15.149982 15.174979
 15.199976 15.224973 15.24997  15.274967 15.299964 15.324961 15.349958
 15.374955 15.399952 15.424949 15.449946 15.474943 15.49994 ]

dH par
[ 0.  0. 30.]

sum of terms
[0.02239954 0.02159325 0.02042496 0.01877513 0.01660934 0.01410984
 0.01166224 0.00960668 0.00805012 0.006924   0.00611174 0.00551383
 0.00506018 0.00470465 0.00441724 0.00417841 0.00397514 0.00379858
 0.00364256 0.00350276]
[0.02239 0.02159 0.02042 0.01878 0.01661 0.01412 0.01167 0.00961 0.00805
 0.00693 0.00611 0.00551 0.00506 0.0047 0.00442 0.00418 0.00397 0.0038
 0.00364 0.0035]
input [3.99975586] 4.527905004025601e-10

eq con
[[9.09090925e-18 1.00000000e-03 9.09090925e-17 8.25981850e-32]
 [1.74661608e-05 8.49671977e-04 1.48405074e-04 2.59061014e-07]
 [4.08990078e-05 7.06532005e-04 2.88964580e-04 1.18117227e-06]
 [7.29609241e-05 5.73524680e-04 4.18448907e-04 3.05132609e-06]



eq con
[[9.09323247e-18 1.00000000e-03 9.09067693e-17 8.27101214e-32]
 [1.74703966e-05 8.49676558e-04 1.48400146e-04 2.59406726e-07]
 [4.09081544e-05 7.06542695e-04 2.88952347e-04 1.18271540e-06]
 [7.29752654e-05 5.73542894e-04 4.18426822e-04 3.05519785e-06]
 [1.17193274e-04 4.54563958e-04 5.32568670e-04 6.24485618e-06]
 [1.77292474e-04 3.53702657e-04 6.26911953e-04 1.11209270e-05]
 [2.55594558e-04 2.73464636e-04 6.98764295e-04 1.78700781e-05]
 [3.51528595e-04 2.13128093e-04 7.48995638e-04 2.63441438e-05]
 [4.62019159e-04 1.69199573e-04 7.81514749e-04 3.61277824e-05]
 [5.83248750e-04 1.37387361e-04 8.01084866e-04 4.67494478e-05]
 [7.11959303e-04 1.14046355e-04 8.11735445e-04 5.78247574e-05]
 [8.45792100e-04 9.65476518e-05 8.16362919e-04 6.90861569e-05]
 [9.83147388e-04 8.31153455e-05 8.16916699e-04 8.03601141e-05]
 [1.12295651e-03 7.25687802e-05 8.14686819e-04 9.15372302e-05]
 [1.26449930e-03 6.41170861e-05 8.10532253e-04 1.02549379e-04]
 [1.40728102e-03 5.72210142e-05 8.05034162e-04 


eq con
[[9.09090925e-18 1.00000000e-03 9.09090925e-17 8.26678638e-32]
 [1.74661120e-05 8.49672145e-04 1.48404688e-04 2.59278157e-07]
 [4.08987288e-05 7.06532707e-04 2.88962896e-04 1.18215374e-06]
 [7.29600078e-05 5.73526270e-04 4.18444811e-04 3.05383191e-06]
 [1.17172200e-04 4.54540313e-04 5.32594884e-04 6.24228574e-06]
 [1.77267009e-04 3.53673067e-04 6.26945667e-04 1.11168025e-05]
 [2.55567614e-04 2.73431777e-04 6.98803069e-04 1.78641638e-05]
 [3.51503431e-04 2.13095077e-04 7.49036506e-04 2.63362917e-05]
 [4.61998086e-04 1.69168589e-04 7.81555644e-04 3.61178717e-05]
 [5.83232834e-04 1.37359368e-04 8.01124935e-04 4.67373716e-05]
 [7.11948760e-04 1.14021495e-04 8.11774622e-04 5.78104404e-05]
 [8.45786731e-04 9.65256895e-05 8.16401473e-04 6.90695645e-05]
 [9.83146835e-04 8.30959276e-05 8.16954982e-04 8.03412494e-05]
 [1.12296038e-03 7.25515492e-05 8.14725154e-04 9.15161265e-05]
 [1.26450723e-03 6.41017218e-05 8.10570906e-04 1.02526091e-04]
 [1.40729265e-03 5.72072438e-05 8.05073338e-04 

In [7]:
print(np.array(c_res_out))
#print(c_res_out[int(np.sum(ser_counts[:0])) + 1])

[[9.09090925e-18 1.00000000e-03 9.09090925e-17 8.26678638e-32]
 [1.74661120e-05 8.49672145e-04 1.48404688e-04 2.59278157e-07]
 [4.08987288e-05 7.06532707e-04 2.88962896e-04 1.18215374e-06]
 [7.29600078e-05 5.73526270e-04 4.18444811e-04 3.05383191e-06]
 [1.17172200e-04 4.54540313e-04 5.32594884e-04 6.24228574e-06]
 [1.77267009e-04 3.53673067e-04 6.26945667e-04 1.11168025e-05]
 [2.55567614e-04 2.73431777e-04 6.98803069e-04 1.78641638e-05]
 [3.51503431e-04 2.13095077e-04 7.49036506e-04 2.63362917e-05]
 [4.61998086e-04 1.69168589e-04 7.81555644e-04 3.61178717e-05]
 [5.83232834e-04 1.37359368e-04 8.01124935e-04 4.67373716e-05]
 [7.11948760e-04 1.14021495e-04 8.11774622e-04 5.78104404e-05]
 [8.45786731e-04 9.65256895e-05 8.16401473e-04 6.90695645e-05]
 [9.83146835e-04 8.30959276e-05 8.16954982e-04 8.03412494e-05]
 [1.12296038e-03 7.25515492e-05 8.14725154e-04 9.15161265e-05]
 [1.26450723e-03 6.41017218e-05 8.10570906e-04 1.02526091e-04]
 [1.40729265e-03 5.72072438e-05 8.05073338e-04 1.133292

In [8]:
#print('\nStoich coeff data')
#print(st_coeff_data)

#print('\nlg K')
#print(lg_k_data)

#print('\nconcentrations')
#print(con_data)

#print('\ntype con')
#print(type_con)

#print('\nexperimental data')
#print(heats_data)

print('\n_calor_setup_type_coeff')
print(_calor_setup_type_coeff)

print('\ntargets')
print(targets)

#print('\ndelta_H')
#print(delta_H)


_calor_setup_type_coeff
[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]

targets
            0   1    2
0  constants   HL  H2L


In [9]:
print('\nSeries')
print(ser_num)
print(ser_counts)
print(ser_info)

print('\nSt coeff matr')
print(st_coeff_matrix)

print('\nprod names')
print(prod_names)
print(prod_names_con)

print('\nconst')
print(lg_k)

print('\nconcentrations')
print(con_matrix)

print(ign_indices)

print('\ntype con')
print(type_con)

print('\nvolumes')
print(volumes)

print('\nheats')
print(heats)

print('\ndeviations')
print(devs)

print('\nwhat to find and what is known')
print(tar_num)
print(dH_ind)
print(dH_ind_wtf)

print('\nenthalpies')
print(dH_par)


Series
1
[21]
['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']

St coeff matr
[[1 0]
 [0 1]
 [1 1]
 [2 1]]

prod names
['H', 'L', 'HL', 'H2L']
['H', 'L']

const
[[0.        ]
 [0.        ]
 [4.        ]
 [6.00024414]]

concentrations
[[1.00000000e-16 1.00000000e-03]
 [1.66389351e-04 9.98336106e-04]
 [3.32225914e-04 9.96677741e-04]
 [4.97512438e-04 9.95024876e-04]
 [6.62251656e-04 9.93377483e-04]
 [8.26446281e-04 9.91735537e-04]
 [9.90099010e-04 9.90099010e-04]
 [1.15321252e-03 9.88467875e-04]
 [1.31578947e-03 9.86842105e-04]
 [1.47783251e-03 9.85221675e-04]
 [1.63934426e-03 9.83606557e-04]
 [1.80032733e-03 9.81996727e-04]
 [1.96078431e-03 9.80392157e-04]
 [2.12071778e-03 9.78792822e-04]
 [2.28013029e-03 9.77198697e-04]
 [2.43902439e-03 9.75609756e-04]
 [2.59740260e-03 9.74025974e-04]
 [2.75526742e-03 9.72447326e-04]
 [2.91262136e-03 9.70873786e-04]
 [3.06946688e-03 9.69305331e-04]
 [3.22580645e-03 9.67741935e-04]]
[]

type con
0    tot
1    tot
Name: 0, dtype: object



In [10]:
print('\nEq conc')
print(np.array(c_res_out))
print(np.shape(c_res_out))


Eq conc
[[9.09090925e-18 1.00000000e-03 9.09090925e-17 8.26678638e-32]
 [1.74661120e-05 8.49672145e-04 1.48404688e-04 2.59278157e-07]
 [4.08987288e-05 7.06532707e-04 2.88962896e-04 1.18215374e-06]
 [7.29600078e-05 5.73526270e-04 4.18444811e-04 3.05383191e-06]
 [1.17172200e-04 4.54540313e-04 5.32594884e-04 6.24228574e-06]
 [1.77267009e-04 3.53673067e-04 6.26945667e-04 1.11168025e-05]
 [2.55567614e-04 2.73431777e-04 6.98803069e-04 1.78641638e-05]
 [3.51503431e-04 2.13095077e-04 7.49036506e-04 2.63362917e-05]
 [4.61998086e-04 1.69168589e-04 7.81555644e-04 3.61178717e-05]
 [5.83232834e-04 1.37359368e-04 8.01124935e-04 4.67373716e-05]
 [7.11948760e-04 1.14021495e-04 8.11774622e-04 5.78104404e-05]
 [8.45786731e-04 9.65256895e-05 8.16401473e-04 6.90695645e-05]
 [9.83146835e-04 8.30959276e-05 8.16954982e-04 8.03412494e-05]
 [1.12296038e-03 7.25515492e-05 8.14725154e-04 9.15161265e-05]
 [1.26450723e-03 6.41017218e-05 8.10570906e-04 1.02526091e-04]
 [1.40729265e-03 5.72072438e-05 8.05073338e-04

In [11]:
print('\nlg K')
print(lg_k)

print('\nF_min')
print(F_min)

print('\nFinal enthalpies')
print(dH_res_out)

print('\ncalculated error of enthalpy')
print(disp)

#if len(tar_num) > 0:
    #print('\ncalculated error of constant')
    #print(errs_values)


lg K
[[0.        ]
 [0.        ]
 [4.        ]
 [6.00024414]]

F_min
4.349948379810846e-10

Final enthalpies
[ 0.          0.          9.99290074 30.        ]

calculated error of enthalpy
[0.00105248]


In [12]:
#import matplotlib.pyplot

#s = 4.5

#xxx = []
#FFF = []
#while s < 5.5:
    #lg_k[tar_num] = s
    
    #c_res_out = eq_calc(max_iter, eps, ser_num, st_coeff_matrix, type_con, lg_k,
        #con_matrix, ign_indices, ser_counts, ser_info)
        
    #F_min, dH_res_out = enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               #_calor_setup_type_coeff)
    
    #xxx.append(s)
    #FFF.append(F_min)
    #s += 0.02


#matplotlib.pyplot.plot(xxx, FFF, 'ro-')

#import sys
#import os
##sys.path.append('../kev_calorim')
#print(os.path.abspath('../kev/'))
#print(os.getcwd())

#lg_k is array

In [13]:
import matplotlib.pyplot as plt
from copy import copy, deepcopy

def risovalka(max_iter, eps, st_coeff_matrix, type_con, lg_k, con_matrix, ign_indices, ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               _calor_setup_type_coeff, _calor_setup_type):

    s = [[0] * 42 for i in range(42)]
    const_x = [0] * 42
    const_y = [0] * 42
    s = np.array(s)

    i, a = 3, 0

    for counter_x in range(len(s)):
        j = 5
        const_x[counter_x] = round(i, 2)
        for counter_y in range(len(s)):
            lg_k[2], lg_k[3] = np.array(i), np.array(j)
            const_y[counter_y] = round(j, 2)
            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                            con_matrix, ign_indices)
        
            F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type)
            
            s[counter_x, counter_y] = float(F_min * 100000000)
        
            j += 0.05
    
        i += 0.05

    s = pd.DataFrame(data = s, index = const_y, columns = const_x)

    fig, ax = plt.subplots(figsize=(20, 20))
    im = ax.imshow(s, cmap = 'RdYlGn_r')
    ax.set_xticks(np.arange(len(const_x)))
    ax.set_yticks(np.arange(len(const_y)))

    ax.set_xlabel('lg K1')
    ax.set_ylabel('lg K2')

    ax.set_xticklabels(const_x)
    ax.set_yticklabels(const_y)

    plt.show()
    return s

In [14]:
#comment this section for reducing the waiting time
#s = risovalka(max_iter, eps, st_coeff_matrix, type_con, lg_k, con_matrix, ign_indices, ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               #_calor_setup_type_coeff, _calor_setup_type)
#with pd.ExcelWriter('d:/KEVreplica/output/calorimetry/test_5.xlsx', mode = "w") as output:
    #s.to_excel(output)

In [15]:
F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff, _calor_setup_type) 


eq con
[[9.09090925e-18 1.00000000e-03 9.09090925e-17 8.26678638e-32]
 [1.74661120e-05 8.49672145e-04 1.48404688e-04 2.59278157e-07]
 [4.08987288e-05 7.06532707e-04 2.88962896e-04 1.18215374e-06]
 [7.29600078e-05 5.73526270e-04 4.18444811e-04 3.05383191e-06]
 [1.17172200e-04 4.54540313e-04 5.32594884e-04 6.24228574e-06]
 [1.77267009e-04 3.53673067e-04 6.26945667e-04 1.11168025e-05]
 [2.55567614e-04 2.73431777e-04 6.98803069e-04 1.78641638e-05]
 [3.51503431e-04 2.13095077e-04 7.49036506e-04 2.63362917e-05]
 [4.61998086e-04 1.69168589e-04 7.81555644e-04 3.61178717e-05]
 [5.83232834e-04 1.37359368e-04 8.01124935e-04 4.67373716e-05]
 [7.11948760e-04 1.14021495e-04 8.11774622e-04 5.78104404e-05]
 [8.45786731e-04 9.65256895e-05 8.16401473e-04 6.90695645e-05]
 [9.83146835e-04 8.30959276e-05 8.16954982e-04 8.03412494e-05]
 [1.12296038e-03 7.25515492e-05 8.14725154e-04 9.15161265e-05]
 [1.26450723e-03 6.41017218e-05 8.10570906e-04 1.02526091e-04]
 [1.40729265e-03 5.72072438e-05 8.05073338e-04 